In [7]:
#import get to call a get request on the site
from requests import get

#get the first page of the housing prices
response = get('https://minneapolis.craigslist.org/search/apa?hasPic=1&availabilityMode=0') #get rid of those lame-o's that post a housing option without a pic using their filter

from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')

#get the macro-container for the housing posts
posts = html_soup.find_all('li', class_= 'result-row')
print(type(posts)) #to double check that I got a ResultSet
print( len(posts)) #to double check I got 120 (elements/page)

<class 'bs4.element.ResultSet'>
120


In [17]:
print("First post: " + posts[0].text.strip())

First post: $1601



favorite this post

May 27
On bus route, Underground heated parking, Balcony

$1601

                    2br -
                    1275ft2 -
                

pic


hide this posting



restore
restore this posting


In [26]:
curtime = posts[0].find('time',class_='result-date')['datetime']
print(curtime)
cururl = posts[0].find('a',class_='result-title hdrlnk')['href']
print(cururl)

2020-05-27 19:59
https://minneapolis.craigslist.org/hnp/apa/d/minneapolis-on-bus-route-underground/7128042130.html
